In [ ]:
import openai
import json
import time
import sys
import os
from datetime import datetime
from pathlib import Path

enableVerboseOutput = False

def load_parameters(node_config):
    global api_key
    api_key = node_config['settings'][0]['api_key']

    global model_name
    model_name = node_config['settings'][0]['model_name']
    global max_tokens
    max_tokens = int(node_config['settings'][0]['max_tokens'])
    global temperature
    temperature = float(node_config['settings'][0]['temperature'])
    global top_p
    top_p = float(node_config['settings'][0]['top_p'])
    global frequency_penalty
    frequency_penalty = float(node_config['settings'][0]['frequency_penalty'])
    global vecstore_path
    vecstore_path = node_config['settings'][0]['vectorstore_path']
    global doc_context_size
    doc_context_size = int(node_config['settings'][0]['document_context_size'])
    global dump_save_path
    dump_save_path = node_config['settings'][0]['dump_save_path']

    # Assemble the LLM input config
    global llm_config
    llm_config = {
                    "temp": temperature,
                    "model": model_name,
                    "top_p": top_p,
                    "freq_penalty": frequency_penalty,
                    "max_tokens": max_tokens
                }

def gpt_inference_direct(question_input: str, config):

    if question_input != "":

        # Grab and print response
        if enableVerboseOutput:
            print("Running inference...")

        response = openai.chat.completions.create(model=config['model'],
                                                max_tokens=config['max_tokens'],
                                                temperature=config['temp'],
                                                top_p=config['top_p'],
                                                frequency_penalty=config['freq_penalty'],
                                                messages=[
                                                            {"role": "user", "content": question_input}
                                                        ])

        # print(response)
        response_str = response.choices[0].message.content
        tokens_used = int(response.usage.total_tokens)

        return response_str, tokens_used

def printDirect(text):
    # Print all text at once to terminal
    print(text)

def printTw(text):
    # Print with typewriter effect
    for char in text:
        sys.stdout.write(char)
        sys.stdout.flush()

        if char != "\n":
            time.sleep(0.1)
        else:
            time.sleep(1)


f = open("config.json",)
config_file = json.load(f)
f.close()

load_parameters(config_file)

# Set-up OpenAI API Key
openai.api_key = api_key
os.environ['OPENAI_API_KEY'] = api_key

# Prompt loop for GPT Neo

last_prompt = ""
last_response = ""

prompt_context = ""
enable_context = False

while True:
    print(" ")
    prompt = input("Human >> ")
    
    if prompt == "quit": break

    if prompt == "clear":
        os.system('clear')
        continue

    if "maxLength=" in prompt:
        tmp_str = prompt.split("=")
        max_length = int(tmp_str[1])
        printDirect(f"--> Max Length set to {str(max_length)}.")
        continue

    if "file=" in prompt:
        # -> Use the content of a file as prompt input and invoke inference
        # Example:
        # file=persona_4G_1.txt
        tmp_str = prompt.split("=")
        fname = tmp_str[1]
        # Read input text file
        if fname:
            # Check if file is present
            if Path("prompts/" + fname).is_file():
                # File existing
                tmp_file_content = ""
                with open("prompts/" + fname, encoding='utf8', mode='r') as f:
                    for line in f:
                        tmp_file_content += f"{line.strip()}\n"
                prompt = tmp_file_content

            else:
                print("File not found.")
                continue

    if "load_context=" in prompt:
        # -> Load file content as instructions and append it to dynamic prompt input
        # Example:
        # file=persona_4G_1.txt
        tmp_str = prompt.split("=")
        fname = tmp_str[1]
        # Read input text file
        if fname:
            # Check if file is present
            if Path("context/" + fname).is_file():
                # File existing
                tmp_file_content = ""
                with open("context/" + fname, encoding='utf8', mode='r') as f:
                    for line in f:
                        tmp_file_content += f"{line.strip()}\n"
                prompt_context = tmp_file_content

                enable_context = True
                print("--> Context loaded and will be applied on each prompt.")
                continue

            else:
                print("File not found.")
                continue

    if "personality_test=" in prompt:
        # Example:
        # personality_test=persona_4G_Test.txt=personality_test_outer.txt
        tmp_str = prompt.split("=")
        print(tmp_str)
        fname_persona = tmp_str[1]
        fname_test_input = tmp_str[2]

        # Read persona file
        if fname_persona:
            # Check if file is present
            if Path("prompts/" + fname_persona).is_file():
                # File existing
                tmp_file_content = ""
                with open("prompts/" + fname_persona, encoding='utf8', mode='r') as f:
                    for line in f:
                        tmp_file_content += f"{line.strip()}\n"
                base_prompt = tmp_file_content
            else:
                print("Persona file not found.")

            # Read in personality test statements
            llm_response_list = []
            if Path("prompts/" + fname_test_input).is_file():
                # File existing
                tmp_file_content = ""
                with open("prompts/" + fname_test_input, encoding='utf8', mode='r') as f:
                    for line in f:
                        test_statement = f"{line.strip()}\n"
                        if test_statement == "\n":
                            continue
                        
                        input_prompt_tmp = base_prompt +  test_statement

                        # Invoke inference
                        text_out, _ = gpt_inference_direct(input_prompt_tmp, llm_config)
                        llm_response_list.append(text_out)
                        time.sleep(2)

                    # Write results to file
                    with open('saved_prompts/personality_test_results.txt', 'w') as f:
                        i = 1
                        for line in llm_response_list:
                            f.write(f"{i}:\n")
                            f.write(f"{line}\n")
                            f.write(" ")

                            i += 1

            else:
                print("Personality test file not found.")

        else:
            print("File not found.")
            continue

        continue

    # Dump entire history in text file
    if prompt == "dump":
        # Generate timestamp
        dateTimeObj = datetime.now()
        timestampStr = dateTimeObj.strftime("%d-%m-%Y-%H-%M-%S")
        # Assemble prompt + response
        dump_tmp = "prompt:\n"
        dump_tmp += last_prompt + "\n"
        dump_tmp += "\nresponse:\n"
        dump_tmp += last_response + "\n"

        with open(f"{dump_save_path}/{timestampStr}.txt", "w") as text_file:
            text_file.write(dump_tmp)
        
        printDirect("--> Prompt + Response saved to text file.")
        continue

    text_out = ""

    # Append context if activated
    if enable_context:
        prompt = prompt_context + "\n" + prompt

    # Inference
    text_out, tokens_used_tmp = gpt_inference_direct(prompt, llm_config)
    last_prompt = prompt
    last_response = text_out

    print(" ")
    sys.stdout.write("GPT >> ")
    sys.stdout.flush()
    # printTw(text_out)
    printDirect(text_out)

    if enableVerboseOutput:
        print(" ")
        printDirect(f"--> Token used for request: {tokens_used_tmp}")

    print(" ")